In [1]:
# Initial imports
print("Importing libraries...")

import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import cv2
from matplotlib import pyplot as plt
import rasterio
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os
import geemap
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
ee.Initialize()

# Import deep learning model
print("Importing NN model...")
model = load_model("sat_classifier.model")
print("Done!")

Importing libraries...
Importing NN model...
Done!


In [2]:
# Functions that will be used later on

# Function to get tilelayer url from earthengine server
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_fetcher = map_id['tile_fetcher']
  return tile_fetcher.url_format
  
# Function to get sizes in Human readable format
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def humansize(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

# covert the lat, lon and array into an image
def toImage(lats,lons,data):
 
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
 
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32) #-9999
    coord_arr = np.zeros([nrows, ncols], dtype=object) #-9999
 
    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
                coord_arr[len(uniqueLats)-1-y,x] = (lats[counter],lons[counter]) # we start from lower left corner
    return arr, coord_arr

def ee2rgb(red,green,blue, ROI):

    # Expand the dimensions of the images so they can be concatenated into 3-D.
    np_arr_b2 = np.expand_dims(blue, 2)
    np_arr_b3 = np.expand_dims(green, 2)
    np_arr_b4 = np.expand_dims(red, 2)

    # Stack the individual bands to make a 3-D array.
    rgb_img = np.concatenate((np_arr_b4, np_arr_b3, np_arr_b2), 2)

    # Scale the data to [0, 255] to show as an RGB image.
    rgb_img_test = (255*((rgb_img - 100)/3500)).astype('uint8')

    # Remove the dimensions that are equal to 1s
    final_rgb_img = np.squeeze(rgb_img_test)

    return final_rgb_img

# Handler for drawings on the map
def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

In [27]:
map1 = geemap.Map(zoom=1)
map1.add_basemap('HYBRID')
map1

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [28]:
# Sentinel-2 MSI: MultiSpectral Instrument, Level-2A dataset
# Link to dataset:
#       https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#description
# Dataset filtered by date from January 1st, 2021 to February 10th, 2021. Also filtered by Cloud cover -> We want clean images
# Filtered by Area of Interest also. 
aoi = ee.FeatureCollection(map1.draw_features)

print("Grabbing image collection...")

img_collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-01-01','2021-2-10').filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',2).filterBounds(aoi)

# Get collection size
print('Number of l8sr assets: '+str(img_collection.size().getInfo()))
print('\n'+'Size l8sr collection : '+str(humansize(img_collection.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))

# Get first image of the collection
sample_image = ee.Image(img_collection.first())
band_names_original = sample_image.bandNames()

Grabbing image collection...
Number of l8sr assets: 19

Size l8sr collection : 29.9 GB


In [29]:
#Create a slider widget to show different bands of the images from the dataset

# Compute the median
img = img_collection.median()

# Compute the NDVI (Vegetation index) -> (Nir-Red)/(Nir+Red)
ndvi = img.normalizedDifference(['B8','B4']).rename('ndvi')

aoi_coordinates = aoi.getInfo().get('features')[0].get('geometry').get('coordinates')
# Create a map centered in the middle of the chosen Rectangle
geometry_mean = np.mean(np.squeeze(np.array(aoi_coordinates)),axis=0)

map2 = ipyleaflet.Map(
    center=(geometry_mean[1], geometry_mean[0]), zoom=12,
    layout={'height':'500px'},
)

# Create the tiles for the map overlay. 
# img_collection_tile_url displays the image with bands B4, B3, B2 -> RGB
# ndvi_tile_url highlights the NDVI
img_collection_tile_url = GetTileLayerUrl(img.visualize(min=100, max=3500, gamma=1.5, bands= ['B4','B3','B2']))  
ndvi_tile_url = GetTileLayerUrl(ndvi.visualize(bands=['ndvi'], min=-0.3, max=0.8, palette='white,c7ffbd,green')) 

left = ipyleaflet.TileLayer(url=img_collection_tile_url)
right=ipyleaflet.TileLayer(url=ndvi_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map2.add_control(control)
map2

Map(center=[37.1295164, -8.557058199999998], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…

In [32]:
# Get the latitudes and longitudes of the image and reduce it to a region
latlon_ee = ee.Image.pixelLonLat().addBands(ndvi)

# apply reducer to list
latlon = latlon_ee.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=aoi,
  scale=10,
  bestEffort=True)

# get data into three different arrays
data = np.array((ee.Array(latlon.get("ndvi")).getInfo()))
lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))

# Transform NDVI earth engine image to a numpy image
npImg, coordImg = toImage(lats,lons,data)

In [33]:
cv2.imshow('npImg_aux', npImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
# Create a copy of the image
img_copy = npImg.copy()

# Multiply by 255 to get an image from 0 to 255 and cast it as uint8
#img_copy[img_copy<0.2] = 0
img_copy = img_copy*255
img_copy = img_copy.astype('uint8')

# Gaussian blur and Threshold. THRESH_OTSU makes the threshold value dynamic
blur = cv2.GaussianBlur(img_copy,(5,5),cv2.BORDER_REFLECT)
_, binary = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [37]:
# getting ROIs with findContours
contours,hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

stacked_img = np.stack((img_copy,)*3, axis=-1)

out_dir = os.getcwd()
out_dir = os.path.join(out_dir,'gee_image_downloads')
filename = os.path.join(out_dir, 'gee_image.tif')

for cnt in contours:
    if ((np.size(cnt)>=100)):# and (np.size(cnt)<=200)):
        (x,y,w,h) = cv2.boundingRect(cnt)

        ROI = img_copy[y:y+h,x:x+w]
        ROI_latlon = coordImg[y:y+h,x:x+w] # ROI_latlon[0][0][0] to get latitude of top left corner

        # Area of interest
        ROI_ee = ee.Geometry.Polygon(
            [[[ROI_latlon[0][0][1],ROI_latlon[-2][0][0]],
                [ROI_latlon[0][-2][1],ROI_latlon[-2][0][0]],
                [ROI_latlon[0][-2][1],ROI_latlon[0][0][0]],
                [ROI_latlon[0][0][1],ROI_latlon[0][0][0]],
                [ROI_latlon[0][0][1],ROI_latlon[-2][0][0]]]], None, False)

        geemap.ee_export_image(img_collection.first(), filename=filename, scale=5, region=ROI_ee, file_per_band=True)

        # Transform earth engine image to the region of interest in rgb
        #rgb_img = ee2rgb(img_collection.first(), ROI_ee)
        rgb_img = ee2rgb(plt.imread(os.path.join(out_dir, 'gee_image.B4.tif')),plt.imread(os.path.join(out_dir, 'gee_image.B3.tif')),plt.imread(os.path.join(out_dir, 'gee_image.B2.tif')), ROI_ee)
        rgb_img_copy = rgb_img.copy()
        rgb_img = cv2.resize(rgb_img, (224, 224))
        rgb_img_model = np.expand_dims(rgb_img, axis=0)

        (crop_field, golf_course) = model.predict(rgb_img_model)[0]

        if crop_field > 0.6:
            label = "c"
            label = "{}:{:.2f}%".format(label, crop_field * 100)
            color = (0, 255, 0)
        elif golf_course > 0.6:
            label = "g"
            label = "{}:{:.2f}%".format(label, golf_course * 100)
            color = (255, 0, 0)
        else:
            label = "idk"
            color = (0, 0, 255)
        
        if (x+rgb_img_copy.shape[1] < stacked_img.shape[1]) and (y+rgb_img_copy.shape[0] < stacked_img.shape[0]):
            cv2.putText(stacked_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(stacked_img, (x, y), (x+rgb_img_copy.shape[1], y+rgb_img_copy.shape[0]), color, 2)
            stacked_img[y:y+rgb_img_copy.shape[0],x:x+rgb_img_copy.shape[1]] = rgb_img_copy

cv2.imshow('npImg_aux', stacked_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desktop\projects\EO_project\gee_image_downloads
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Ivan\Desk